In [ ]:

# Google earth engine
import ee
# Authenticate and initialize Earth Engine API
ee.Authenticate()  # Only required once
ee.Initialize(
      opt_url='https://earthengine-highvolume.googleapis.com'
)

# Built-in modules
import os
import subprocess
from pathlib import Path
import datetime
from datetime import date
from urllib.request import urlopen
from typing import Any, Optional, Tuple

# Basics of Python data handling and visualization
import math
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# Geospatial
import geemap
import geopandas as gpd
from shapely import wkt
from shapely.geometry import (
    shape,
    Point,
    LineString,
    Polygon,
    MultiPoint,
    MultiLineString,
    MultiPolygon
)
from shapely.validation import make_valid
import geojson
import folium
from geopy.distance import great_circle
from gadm import GADMDownloader
import rasterio
from rasterio import plot
import xarray as xr  # It may need Dask library https://docs.dask.org/en/stable/install.html
import rioxarray  # noqa: F401 # Its necesary for xarray.open_mfdataset() with engine `rasterio`


# The following is not a package. It is a file utils.py which should be in the same folder as this notebook.
from utils import (
    get_adjacent_blocks, 
    get_new_adjacent_blocks,
    plot_block_and_adjacent
)


In [ ]:
# GLOBAL PARAMS:

# Country: Select one of: ["Brazil", "India", "Indonesia", "Ethiopia", "Nigeria"]
COUNTRY = "Nigeria"

# FILE PATHS (GLOBALS)
cdir = os.getcwd()
pdir = os.path.dirname(cdir)
OUTPUTS_FOLDER = os.path.join(pdir, "outputs", COUNTRY)

In [ ]:
# Open sampling grid for selected country

sampling_grid_final = os.path.join(OUTPUTS_FOLDER, "sampling_grid_final.geojson")
# Convert to geopandas dataframe
gdf_sampling_grid = gpd.read_file(sampling_grid_final)

In [ ]:
# Testing functions that plot the block and its adjacent blocks

# Randomly select a single block from built_blocks_gdf
block_id = sampling_grid_final.sample(n=1, random_state=prng)['block_id'].values[0]

# Plot the block and its adjacent blocks
plot_block_and_adjacent(block_id, sampling_grid_final)


In [ ]:
# Return adjancet blocks in a second expansion
print(block_id)
origin = sampling_grid_final[sampling_grid_final['block_id'] == block_id]
adjacent_blocks = get_adjacent_blocks(block_id, sampling_grid_final)
network = pd.concat([origin, adjacent_blocks])

edge_block_id = "14953"
new_adjacenet_blocks = get_new_adjacent_blocks(edge_block_id, network, sampling_grid_final)
new_network = pd.concat([network, new_adjacenet_blocks])

# Plot the network
fig, ax = plt.subplots(figsize=(10, 10))
# Plot the block
origin.boundary.plot(ax=ax, color="green", linewidth = 1.75, aspect = 1)
# Plot the adjacent blocks
adjacent_blocks.boundary.plot(ax=ax, color="orange", linewidth = 0.75)
# Plot the new adjacent blocks
new_adjacenet_blocks.boundary.plot(ax=ax, color="red", linewidth = 0.75)

In [ ]:
####################################################################
# VISUALIZE NETWORK
###################################################################

# Converting assets to ee objects
origin_ee = geemap.gdf_to_ee(origin)
adjacent_blocks_ee = geemap.gdf_to_ee(adjacent_blocks)
new_adjacenet_blocks_ee = geemap.gdf_to_ee(new_adjacenet_blocks)
network_ee = geemap.gdf_to_ee(network)

m = geemap.Map()
m.centerObject(network_ee, 14)

#############################
# Display the cloudless mosaic
#############################

m.add_layer(
    srh_s2_image,
    {'min': 0,  'max': 2500, 'gamma': 1.1,
     'bands': ['B4', 'B3', 'B2']},
    'S2 cloud-free mosaic',
)

########################
# BLOCK NETWORK
########################

# Add origin block to the map
m.add_layer(origin_ee, {'color': 'green', "width": 2}, "Origin block")
m.add_layer(adjacent_blocks_ee, {'color': 'orange', "width": 2}, "1st expansion")
m.add_layer(new_adjacenet_blocks_ee, {'color': 'red', "width": 2}, "2nd expansion")

m